In [21]:
import pandas as pd

In [22]:
df = pd.read_csv('../data/IMDB/IMDB_Dataset.csv')

df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


# 制作数据集

In [23]:
pos = df[df['sentiment']=='positive'].sample(frac=1.0).drop(columns=['sentiment'])
pos['rating'] = 1
pos

,review,rating
3507,"Well, this is new...Famous Italian horror dire...",1
13399,"In ""Die Nibelungen: Siegfried"", Siegfried was ...",1
21868,"""Sharky's Machine"" is clearly a Burt Reynolds ...",1
11586,"As winter approaches, our state-owned broadcas...",1
14395,Perhaps the best movie ever made by director K...,1
...,...,...
15052,I saw this movie being a Jane Austen addicted ...,1
34174,"Gulliver's Travels is, at the beginning, a sat...",1
6951,Track Listing: 1. Spiderbait - Outta My Head 2...,1
25975,If it had been made 2 years later it would hav...,1


In [24]:
neg = df[df['sentiment']!='positive'].sample(frac=1.0).drop(columns=['sentiment'])
neg['rating'] = 0
neg

,review,rating
33324,I still haven't gotten to see all of this; it'...,0
135,"This film could have been a decent re-make, an...",0
16052,OK..this movie could have been soooo good! All...,0
41110,Ram Gopal Verma has proved himself as a very i...,0
31916,I give it a 2 - I reserve a 1 rating for Guy R...,0
...,...,...
17952,Only three words are really required for this ...,0
26372,This movie was a disappointment. I was looking...,0
9523,While studying the differences between religio...,0
25839,Fairly appalling enterprise suggests Welsh to ...,0


In [25]:
cut_idx1 = int(round(0.7 * pos.shape[0]))
cut_idx2 = int(round(0.85 * pos.shape[0]))
print(cut_idx1)
print(cut_idx2)

17500
21250


In [26]:
train = pd.concat([pos[0:cut_idx1],neg[0:cut_idx1]])
train.shape

(35000, 2)

In [27]:
val = pd.concat([pos[cut_idx1:cut_idx2],neg[cut_idx1:cut_idx2]])
val.shape

(7500, 2)

In [28]:
test = pd.concat([pos[cut_idx2:],neg[cut_idx2:]])
test.shape

(7500, 2)

In [ ]:
train.to_csv('../data/IMDB/train.csv',encoding='utf_8_sig',index=False)
val.to_csv('../data/IMDB/val.csv',encoding='utf_8_sig',index=False)
test.to_csv('../data/IMDB/test.csv',encoding='utf_8_sig',index=False)

In [ ]:
df

# 分析

In [4]:
import pandas as pd
df = pd.read_csv('../data/IMDB/train.csv')

df

,review,rating
0,The Hospital is a movie that was made ahead of...,1
1,The world of the Dragon Hunters is a 3D gravit...,1
2,Who wouldn't want to go on road trip with Pete...,1
3,-Kidnappings in Mexico are as common as honeys...,1
4,"Cary Grant, Douglas Fairbanks Jr. and Victor M...",1
...,...,...
34995,OK - I ADORE this film...I will credit this mo...,0
34996,"Maaan, where do i start with this god awful mo...",0
34997,This really is a film of two halves. The first...,0
34998,"Its difficult to be too tough on Brad Sykes, a...",0


In [14]:
from nltk import tokenize
import re
def getTokens(text):
    text = re.sub("<br />|\'s|\'", "", text)
    return len(tokenize.word_tokenize(text))

In [15]:
import os
from tqdm.auto import tqdm
from joblib import Parallel, delayed



In [16]:
df['tokensNum'] = Parallel(n_jobs=os.cpu_count())(delayed(getTokens)(data) for data in tqdm(df['review']))


  0%|          | 0/35000 [00:00<?, ?it/s]

In [5]:
df[df['tokensNum']<400].shape[0] / df.shape[0]

0.8408857142857142

In [19]:
df[df['tokensNum']<500].shape[0] / df.shape[0]

0.9004571428571428

## check data!

In [ ]:
index = 0
text = re.sub("<br />|\'s|\'", "", df.iloc[index]['review'])
# text = re.sub("", "", text)

text

# Test for T-E-GRU

In [3]:
import gensim.downloader as api
import gensim
word2Vec_Model = gensim.models.KeyedVectors.load_word2vec_format( '../models/embeddings/glove-wiki-gigaword-300.gz')


In [9]:
len(word2Vec_Model.vocab)

400000

In [12]:
word2Vec_Model.vocab['unk'].index


201534

In [4]:
from nltk import tokenize
import re
def getTokensDes(text):
    text = re.sub("<br />|\'s|\'", "", text)
    outOfNum = 0
    tokens = tokenize.word_tokenize(text)
    for word in tokens:
        print(word)
        try:
            word2Vec_Model.vocab[word].index
        except:
            outOfNum += 1
    return len(tokens),outOfNum
getTokensDes('hello world !')

hello
world
!


(3, 0)

In [9]:
import os
from tqdm.auto import tqdm
from joblib import Parallel, delayed
sumOfTokens = 0
outOfNum = 0
for text in tqdm(df['review']):
    text = re.sub("<br />|\'s|\'", "", text)
    tokens = tokenize.word_tokenize(text)
    for word in tokens:
        try:
            word2Vec_Model.vocab[word].index
        except:
            outOfNum += 1
    sumOfTokens += len(tokens) 
    
# df[['NumOfTokens','outOfNum']] = pd.DataFrame(Parallel(n_jobs=os.cpu_count())(delayed(getTokensDes)(data) for data in tqdm(df['review'])))

  0%|          | 0/35000 [00:00<?, ?it/s]

In [10]:
1-outOfNum/sumOfTokens

0.8765053249666424

87.7

# Test for BERT

In [5]:
from transformers import AutoTokenizer
Pre_TrainPath = "../models/bert-base-cased/"

tokenizer = AutoTokenizer.from_pretrained(Pre_TrainPath)

In [6]:
import re
def getTokenslen(text):
    text = re.sub("<br />|\'s|\'", "", text)
    tokens = tokenizer(text, return_tensors='pt',return_length=True,return_token_type_ids=False,return_attention_mask=False)
    return tokens['length'].item(),(tokens['input_ids']==100).sum().item()
getTokenslen('hello world !')

(5, 0)

In [7]:
from tqdm.auto import tqdm
sumOfTokens = 0
outOfNum = 0
tokensLength = []
for text in tqdm(df['review']):
    tempLen = getTokenslen(text)
    tokensLength.append(tempLen)
    sumOfTokens += tempLen[0]
    outOfNum += tempLen[1] 


  0%|          | 0/35000 [00:00<?, ?it/s]

In [8]:
1-outOfNum/sumOfTokens

0.9999992249328259

In [9]:
df[['NumOfTokens','outOfNum']] = pd.DataFrame(tokensLength)

In [10]:
df[df['NumOfTokens']<510].shape[0] / df.shape[0]

0.8707142857142857

In [11]:
df

,review,rating,NumOfTokens,outOfNum
0,The Hospital is a movie that was made ahead of...,1,761,0
1,The world of the Dragon Hunters is a 3D gravit...,1,262,0
2,Who wouldn't want to go on road trip with Pete...,1,230,0
3,-Kidnappings in Mexico are as common as honeys...,1,832,0
4,"Cary Grant, Douglas Fairbanks Jr. and Victor M...",1,123,0
...,...,...,...,...
34995,OK - I ADORE this film...I will credit this mo...,0,184,0
34996,"Maaan, where do i start with this god awful mo...",0,184,0
34997,This really is a film of two halves. The first...,0,406,0
34998,"Its difficult to be too tough on Brad Sykes, a...",0,199,0


In [ ]:

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# tokenized_datasets = dataset.map(tokenize_function, batched=True)


In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(Pre_TrainPath, num_labels=2)

In [ ]:
maxLength = model.bert.embeddings.position_embeddings.num_embeddings
tokenizer = AutoTokenizer.from_pretrained(Pre_TrainPath,padding_side='left',truncation_side='left')
encoded_input  = tokenizer(text, return_tensors='pt',max_length=maxLength,padding='max_length',truncation=True)
encoded_input['input_ids'] .shape

In [ ]:
model(**encoded_input)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [ ]:
training_args